In [1]:
from audio_preprocessing.cut_audio_segment_from_audio_file import AudioProcessor
from feature_extraction.feature_extractor import FeatureExtractor
from feature_extraction.mfcc_images_extractor import MelSpectrogramImageExtractor
from data_processing.english_data_processing import ModelEvaluator
from itertools import combinations
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [2]:
def extract_mfcc_images(n_mfcc, figsize, input_folder, output_folder):
    mfccImageExtractor = MelSpectrogramImageExtractor(n_mfcc, figsize)
    mfccImageExtractor.extract_from_folders(input_folder, output_folder)

In [3]:
def cut_audio_segment_from_audio_file(frame_size, hop_size, threshold, input_folder, output_folder):
    segmentation = AudioProcessor(frame_size, hop_size, threshold)
    segmentation.process_folders(input_folder, output_folder)

In [4]:
def extract_feature(n_mfcc, input_folder, output_folder, selected_features):
    featureExtractor = FeatureExtractor(n_mfcc)
    featureExtractor.process_folder(input_folder, output_folder, selected_features)
    # featureExtractor.process_folder(input_folder, output_folder, ['mfcc','mfcc_statistics', 'zcr', 'pitch', 'rms'])

In [5]:
def model(featured_data):
    models = [RandomForestClassifier, ExtraTreesClassifier]
    
    df_train_test = pd.read_csv(featured_data)
    df_shuffled = df_train_test.sample(frac=1, random_state=42).reset_index(drop=True)

    modelEvaluator = ModelEvaluator(models)
    best_model, best_scaler, best_avg_acc = modelEvaluator.evaluate_models(df_shuffled)
    return best_model, best_scaler, best_avg_acc

In [6]:
def process_data(frame_size, hop_size, threshold, raw_folder, cutted_folder, n_mfcc_list, featured_data_path, n_mfcc_for_images, images_folder, features):
    n = len(features)

    results = []

    for n_mfcc in n_mfcc_list:
        for r in range(0, n):
            for combo in combinations(range(n), r):
                selected_features = [features[i] for i in range(n) if i not in combo]
                print(selected_features)
                extract_feature(n_mfcc, cutted_folder, featured_data_path, selected_features)
                current_model, current_scaler, current_avg_acc = model(featured_data_path)
                print(f"Scaler: {current_scaler}, Model: {current_model}, n_mfcc: {n_mfcc}, Selected features: {selected_features}, Accuracy: {current_avg_acc}")
                results.append({'Scaler': current_scaler, 'Model': current_model, 'N_MFCC': n_mfcc, 'Selected Features': selected_features, 'Average Accuracy': current_avg_acc})

    results_df = pd.DataFrame(results)
    ranked_results = results_df.sort_values(by='Average Accuracy', ascending=False)

    return ranked_results

In [7]:
#'mfcc','mfcc_statistics', 'zcr', 'pitch', 'rms'
ranked_results = process_data(256, 128, 0.001, 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/combined_data', 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/combined_data', [26], 'D:/data_analysis/speech_emotion_recognition/notebooks/feature1.csv', 100, 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/images', ['mfcc_statistics', 'zcr', 'pitch', 'rms'])

2024-05-23 03:23:18.530 | INFO     | feature_extraction.feature_extractor:process_folder:148 - Processing input folder: D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/combined_data


['mfcc_statistics', 'zcr', 'pitch', 'rms']


2024-05-23 03:25:48.685 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature1.csv
2024-05-23 03:25:48.889 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-23 03:27:18.363 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.633006244424621
2024-05-23 03:27:36.305 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.6339875111507582
2024-05-23 03:29:05.464 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.6322033898305085
2024-05-23 03:29:22.384 | INFO     | data_

Scaler: RobustScaler, Model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>, n_mfcc: 26, Selected features: ['mfcc_statistics', 'zcr', 'pitch', 'rms'], Accuracy: 0.6372881355932203
['zcr', 'pitch', 'rms']


2024-05-23 03:39:02.398 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature1.csv
2024-05-23 03:39:02.424 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-23 03:39:34.662 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.5272078501338091
2024-05-23 03:39:45.391 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.5254237288135594
2024-05-23 03:40:10.145 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.5272078501338091
2024-05-23 03:40:18.311 | INFO     | data

...

Scaler: None, Model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>, n_mfcc: 26, Selected features: ['pitch'], Accuracy: 0.21097234611953614
['zcr']


2024-05-23 05:09:56.907 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature1.csv
2024-05-23 05:09:56.922 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-23 05:10:12.500 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.3868867082961641
2024-05-23 05:10:19.709 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.3813559322033898
2024-05-23 05:10:35.262 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.3846565566458519
2024-05-23 05:10:42.516 | INFO     | data

Scaler: RobustScaler, Model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>, n_mfcc: 26, Selected features: ['zcr'], Accuracy: 0.3870651204281891
['mfcc_statistics']


2024-05-23 05:13:06.933 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature1.csv
2024-05-23 05:13:07.066 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-23 05:14:34.635 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.6264942016057091
2024-05-23 05:14:50.879 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.623728813559322
2024-05-23 05:16:26.213 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.6220338983050848
2024-05-23 05:16:50.265 | INFO     | data_

Scaler: None, Model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>, n_mfcc: 26, Selected features: ['mfcc_statistics'], Accuracy: 0.6264942016057091


In [11]:
ranked_results.head(30)

,Scaler,Model,N_MFCC,Selected Features,Average Accuracy
0,RobustScaler,<class 'sklearn.ensemble._forest.RandomForestC...,26,"[mfcc_statistics, zcr, pitch, rms]",0.637288
2,None,<class 'sklearn.ensemble._forest.RandomForestC...,26,"[mfcc_statistics, pitch, rms]",0.635147
4,RobustScaler,<class 'sklearn.ensemble._forest.RandomForestC...,26,"[mfcc_statistics, zcr, pitch]",0.634701
3,RobustScaler,<class 'sklearn.ensemble._forest.RandomForestC...,26,"[mfcc_statistics, zcr, rms]",0.633720
9,None,<class 'sklearn.ensemble._forest.RandomForestC...,26,"[mfcc_statistics, pitch]",0.631668
8,RobustScaler,<class 'sklearn.ensemble._forest.RandomForestC...,26,"[mfcc_statistics, rms]",0.630419
10,None,<class 'sklearn.ensemble._forest.RandomForestC...,26,"[mfcc_statistics, zcr]",0.627565
14,None,<class 'sklearn.ensemble._forest.RandomForestC...,26,[mfcc_statistics],0.626494
1,MinMaxScaler,<class 'sklearn.ensemble._forest.ExtraTreesCla...,26,"[zcr, pitch, rms]",0.530419
6,None,<class 'sklearn.ensemble._forest.ExtraTreesCla...,26,"[zcr, rms]",0.515968
